In [81]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from numpy import unique
#splitting Train and Test
from sklearn.model_selection import train_test_split

#import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [85]:
#import Data
Data = pd.read_csv('data.csv')
Data.head(5)

Data['Price movement'] = Data['Price movement'].apply(lambda x: 0 if x == 'Down' else 1)

X = Data.drop('Price movement', axis=1).values
y = Data['Price movement'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

print(X_train.shape)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
print(X_train.shape)

print(unique(y))
print(unique(y).sum())

(3779, 2)
(3779, 2, 1)
[0 1]
1


In [93]:
model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(2, 1)))
model.add(Dense(16, activation="relu"))
# model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_31 (Conv1D)          (None, 1, 64)             192       
                                                                 
 dense_34 (Dense)            (None, 1, 16)             1040      
                                                                 
 flatten_15 (Flatten)        (None, 16)                0         
                                                                 
 dense_35 (Dense)            (None, 1)                 17        
                                                                 
Total params: 1,249
Trainable params: 1,249
Non-trainable params: 0
_________________________________________________________________


In [94]:
model.fit(X_train, y_train, batch_size=16,epochs=100, verbose=0)

UnimplementedError:  Cast string to float is not supported
	 [[node sequential_31/Cast
 (defined at c:\users\hansheng\onedrive\desktop\stock-price-classification\venv\lib\site-packages\keras\engine\functional.py:671)
]] [Op:__inference_train_function_56618]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_31/Cast:
In[0] IteratorGetNext (defined at c:\users\hansheng\onedrive\desktop\stock-price-classification\venv\lib\site-packages\keras\engine\training.py:866)

Operation defined at: (most recent call last)
>>>   File "C:\Users\hansheng\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "C:\Users\hansheng\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "c:\users\hansheng\onedrive\desktop\stock-price-classification\venv\lib\site-packages\ipykernel_launcher